# DMRG for Spin Hamiltonian

In [ ]:
using ITensors
using Plots

Define system parameters

In [ ]:
N = 30 # System size
J = 1; # Hopping parameters
PBC = 0; # Periodic boundary conditions

# Define N S=1/2 spin indices
sites = siteinds("S=1/2",N);

Function to define MPO of Hamiltonian

In [ ]:
function Hamiltonian(N,sites,J,PBC)

    # Initiate construction
    ops = OpSum()

    # Hopping terms
    for j=1:N-1
        ops += J,"S+",j,"S-",j+1
        ops += J,"S-",j,"S+",j+1    
    end

    # PBC if required
    if PBC == 1
        ops += J,"S+",1,"S-",N
        ops += J,"S-",1,"S+",N     
    end

    H = MPO(ops,sites);
    
    return H;
    
end;

Function to prepare initial state MPS

In [ ]:
function InitialState(sites,N)
    
    state = [isodd(n) ? "Up" : "Dn" for n=1:N];
    ψi = MPS(sites,state);
    
    return ψi;
end

Function to calculate ground state

In [ ]:
function GroundState(H,ψi)

    # Do many sweeps of DMRG, gradually increasing the maximum MPS bond dimension
    nsweeps = 30;
    maxdim = 100;
    cutoff = 1E-8;

    # Run the DMRG algorithm for ground state   
    energy0,ψ0 = dmrg(H,ψi; nsweeps, maxdim, cutoff, outputlevel=1);

    @show energy0

    # Orthonormalize state. Leave site 1 as orthogonalization center
    orthogonalize!(ψ0,1)

    return energy0, ψ0;
    
end

Call functions to obtain ground state and observables

In [ ]:
# Define spin Hamiltonian
H = Hamiltonian(N,sites,J,PBC);

In [ ]:
# Define initial state
ψi = InitialState(sites,N);

In [ ]:
# Calculate ground state
E0,ψ0 = GroundState(H,ψi);

In [ ]:
# Obtain energy from the ground state and Hamiltonian

# Form 1
Hψ0 = apply(H,ψ0);
E = inner(ψ0,Hψ0);
@show maxlinkdim(ψ0);
@show maxlinkdim(Hψ0);

# Form 2
E = inner(ψ0',H,ψ0);
@show E;

In [ ]:
# Check convergence of ground state to an eigenstate by calculating variance
H2 = inner(H,ψ0,H,ψ0)
var = H2-E^2
@show var

In [ ]:
# Calculate ground state expectation values
Norm = real(norm(ψ0)); # Norm of the state 
Sx = real(expect(ψ0, "Sx"))/Norm^2; # Magnetization along x
Sz = real(expect(ψ0, "Sz"))/Norm^2; # Magnetization along z
SxSx = correlation_matrix(ψ0,"Sx","Sx")/Norm^2; # XX Correlations
SzSz = correlation_matrix(ψ0,"Sz","Sz")/Norm^2; # ZZ Correlations

In [ ]:
plot(Sx, xlabel = "j", ylabel = "Sx(j)")

In [ ]:
heatmap(SxSx, xlabel = "j", ylabel = "k")

Simulation Finished!!!